In [1]:
import spacy
import re

In [2]:
with open('phoner_covid.conll', 'r', encoding='utf-8') as f:
    data = f.read().split('\n\n')
    
data[0]

'Đồng O\nthời O\n, O\nbệnh O\nviện O\ntiếp O\ntục O\nthực O\nhiện O\ncác O\nbiện O\npháp O\nphòng O\nchống O\ndịch O\nbệnh O\nCOVID O\n- O\n19 O\ntheo O\nhướng O\ndẫn O\ncủa O\nBộ B-ORGANIZATION\nY I-ORGANIZATION\ntế I-ORGANIZATION\n. O'

In [3]:
tokens = []
iob_labels = []
for token in data[3].split("\n"):
    word, iob_label = token.split()
    tokens.append(word)
    iob_labels.append(iob_label)
    
print(tokens)
print(iob_labels)

['Bà', 'này', 'khi', 'trở', 'về', 'quá', 'cảnh', 'Doha', '(', 'Qatar', ')', ',', 'đáp', 'xuống', 'Tân', 'Sơn', 'Nhất', 'sáng', '2/3', 'cùng', '75', 'hành', 'khách', ',', 'trong', 'đó', 'có', '55', 'người', 'nước', 'ngoài', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'O', 'B-DATE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [4]:
def get_ents(tokens, iob_labels):
    ents = []

    ent = []
    last_label = ''
    for token, iob_label in zip(tokens, iob_labels):
        if iob_label == 'O':
            continue
        
        iob, label = iob_label.split('-')
        if iob == 'B':
            if last_label:
                ents.append((' '.join(ent), last_label))
            last_label = label
            ent = []
            ent.append(token)
        elif iob == 'I':
            ent.append(token)
            
    ents.append((' '.join(ent), last_label))

    
    return ents

get_ents(tokens, iob_labels)

[('Doha', 'LOCATION'),
 ('Qatar', 'LOCATION'),
 ('Tân Sơn Nhất', 'LOCATION'),
 ('2/3', 'DATE')]

In [5]:
corpus = []
for example in data:
    tokens = []
    iob_labels = []
    for token in example.split("\n"):
        try:
            word, iob_label = token.split()
        except:
            break
        tokens.append(word)
        iob_labels.append(iob_label)
    corpus.append((tokens, iob_labels))

In [6]:
print(corpus[0])

(['Đồng', 'thời', ',', 'bệnh', 'viện', 'tiếp', 'tục', 'thực', 'hiện', 'các', 'biện', 'pháp', 'phòng', 'chống', 'dịch', 'bệnh', 'COVID', '-', '19', 'theo', 'hướng', 'dẫn', 'của', 'Bộ', 'Y', 'tế', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'O'])


In [20]:
LABEL_MAP = {
    "ORGANIZATION": "ORG",
    "NAME": "PERSON_NAME",
    "JOB": "JOB_TITLE",
    "LOCATION": "ADDRESS",
    "DATE": "DATE"
}
nlp = spacy.blank('en')

def create_doc(text, ents):
    doc = nlp(text)
    ents_spacy = []
    for ent in ents:
        if ent[1] not in LABEL_MAP.keys():
            continue
        span = re.search(ent[0], text)
        if span:
            span = span.span()
            e  = doc.char_span(span[0], span[1], LABEL_MAP.get(ent[1]), alignment_mode='contract')
            if e:
                ents_spacy.append(e)
    if ents_spacy:
        ents_spacy = spacy.util.filter_spans(ents_spacy)
        doc.set_ents(ents_spacy)
    
    return doc
        
        

        

In [21]:
db = spacy.tokens.DocBin()
labels_ = []
for tokens, iob_labels in corpus:
    text = ' '.join(tokens)
    ents = get_ents(tokens, iob_labels)
    for e in ents:
        labels_.append(e[1])
    
    doc = create_doc(text, ents)
    db.add(doc)

print(set(labels_))

db.to_disk('phoner_covid.spacy')
    

{'', 'SYMPTOM_AND_DISEASE', 'TRANSPORTATION', 'PATIENT_ID', 'AGE', 'GENDER', 'LOCATION', 'DATE', 'NAME', 'JOB', 'ORGANIZATION'}
